## Traitement du signal

### RAW DATA VISUALIZATION

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neurokit2 as nk

def _raw_to_eda_uS(raw_data):
    VCC = 3.3
    n = 10  # Set n to 10 for all files
    eda_uS = raw_data * VCC
    eda_uS = eda_uS / (0.132 * (2**n))
    return eda_uS

# Define the folder path
folder_path = os.path.join(os.getcwd(), "EDA_Signal_Brut")

# List of file IDs
file_ids = [
    1, 2, 3, 4, 6, 9, 10, 14, 15, 16, 20, 25, 26, 28, 29, 30, 31, 35, 36, 37, 39, 40,
    41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67,
    70, 71, 72, 73, 75, 76, 78, 81, 83, 86, 87, 88, 107, 110, 90, 93, 94, 96, 98, 99,
    101, 103, 104, 105, 106, 109, 112, 113, 114
]

# Files where channel is A3 instead of A6
channel_a3_ids = [20, 41, 88, 98]

# Sampling rate (assume constant for all files)
sampling_rate = 1000

# Process each file
for file_id in file_ids:
    file_name = f"ID{file_id}.txt"
    file_path = os.path.join(folder_path, file_name)

    try:
        # Read the file into a DataFrame
        data = pd.read_csv(file_path, sep="\t", comment="#", header=None)

        # Determine the correct channel
        channel = "A3" if file_id in channel_a3_ids else "A6"

        # Extract the EDA data (fourth column is where data resides)
        raw_eda = data[5]  # Column index 5 corresponds to the EDA data (A6 or A3)

        # Convert raw EDA data to microSiemens (uS)
        eda_uS = _raw_to_eda_uS(raw_eda)

        # Create a DataFrame for plotting
        dataSet = pd.DataFrame({"EDA": eda_uS})

        # Plot raw EDA data
        plt.figure(figsize=(14, 7))
        sns.lineplot(data=dataSet, x=dataSet.index / sampling_rate, y="EDA")
        plt.title(f"Raw EDA over Time - {file_name}")
        plt.xlabel("Time (s)")
        plt.ylabel("EDA (µS)")
        plt.show()

        # Process and plot cleaned EDA data using NeuroKit
        eda_signals, _ = nk.eda_process(dataSet["EDA"], sampling_rate=sampling_rate)
        plt.figure(figsize=(14, 7))
        sns.lineplot(data=eda_signals, x=eda_signals.index / sampling_rate, y="EDA_Clean")
        plt.title(f"Cleaned EDA over Time - {file_name}")
        plt.xlabel("Time (s)")
        plt.ylabel("EDA (µS)")
        plt.show()

    except FileNotFoundError:
        print(f"File {file_name} not found.")
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")


### CLEANED DATA VISUALIZATION

In [ ]:
from EDA.edaanalysis import EdaAnalysis as EA
import os
import pandas as pd
import numpy as np

# Define the folder path for processed EDA files
folder_path = os.path.join(os.getcwd(), "EDA_Converted")

# List of file IDs
file_ids = [
    1, 2, 3, 4, 6, 9, 10, 14, 15, 16, 20, 25, 26, 28, 29, 30, 31, 35, 36, 37, 39, 40,
    41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67,
    70, 71, 72, 73, 75, 76, 78, 81, 83, 86, 87, 88, 107, 110, 90, 93, 94, 96, 98, 99,
    101, 103, 104, 105, 106, 109, 112, 113, 114
]

# Sampling rate
sampling_rate = 1000

# Analyze each file
for file_id in file_ids:
    file_name = f"ID{file_id}_processed.csv"
    file_path = os.path.join(folder_path, file_name)
    
    print(f"Processing ID: {file_id}")

    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_name}")
        continue

    # Load the data
    data = pd.read_csv(file_path)
    
    try:
        if file_id == 9 or file_id == 86:
            # Split EDA into phasic and tonic components
            split_eda = EA.split_phasic_tonic_signals(
            data['EDA (µS)'], 
            phasic_cutoff=0.005 ,   
            sampling_freq=sampling_rate, 
            show_signal_plot=True, 
            show_freq_spectrum=False
        )
        else:
            # Split EDA into phasic and tonic components
            split_eda = EA.split_phasic_tonic_signals(
            data['EDA (µS)'], 
            phasic_cutoff=0.02 ,  # I tried with 0.005, 0,05, 0.02, 0.002(PAS BON)
            sampling_freq=sampling_rate, 
            show_signal_plot=True, 
            show_freq_spectrum=False
        )



        # Detect peaks in the phasic component
        peaks = EA.get_phasic_peaks(
            split_eda['EDA_Phasic'], 
            sampling_freq=sampling_rate, 
            print_summary=True, 
            show_plot=True
        )

        # Combine results into a single DataFrame
        processed_eda = pd.concat([split_eda, peaks], axis=1)
        

        # Calculate phasic features
        print(f"Phasic Peak Count: {processed_eda['SCR_Peaks'].sum()}")
        print(f"Phasic Mean Amplitude: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_Amplitude'].mean()}")
        print(f"Phasic Max Amplitude: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_Amplitude'].max()}")
        print(f"Phasic Min Amplitude: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_Amplitude'].min()}")
        print(f"Phasic STD Amplitude: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_Amplitude'].std()}")
        print(f"Phasic Sum Amplitude: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_Amplitude'].sum()}")
        print(f"Phasic Mean RiseTime: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_RiseTime'].mean()}")
        print(f"Phasic Max RiseTime: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_RiseTime'].max()}")
        print(f"Phasic Min RiseTime: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_RiseTime'].min()}")
        print(f"Phasic STD RiseTime: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_RiseTime'].std()}")
        print(f"Phasic Sum RiseTime: {processed_eda[processed_eda['SCR_Peaks'] > 0]['SCR_RiseTime'].sum()}")
        print(f"Phasic AUC: {np.trapz(np.abs(processed_eda['EDA_Phasic']), processed_eda['t_sec'])}")

        # Calculate tonic features
        print(f"Tonic Mean: {processed_eda['EDA_Tonic'].mean()}")
        print(f"Tonic STD: {processed_eda['EDA_Tonic'].std()}")
        print(f"Tonic Variance: {processed_eda['EDA_Tonic'].var()}")
        print(f"Tonic Max: {processed_eda['EDA_Tonic'].max()}")
        print(f"Tonic Min: {processed_eda['EDA_Tonic'].min()}")

        # Compute percentage of increasing and decreasing changes
        def compute_changes(curve):
            diff = np.diff(curve)
            ic = len(diff[diff > 0])
            dc = len(diff[diff < 0])
            tc = ic + dc
            return (ic / tc) * 100, (dc / tc) * 100

        ip, dp = compute_changes(processed_eda['EDA_Tonic'])
        print(f"Tonic: Percentage of times increasing: {ip}")
        print(f"Tonic: Percentage of times decreasing: {dp}")


        # Calculate the total number of points
        total_points = len(processed_eda['EDA_Phasic'])

        # Calculate the number of points below 0
        points_below_zero = (processed_eda['EDA_Phasic'] < 0).sum()

        # Calculate the percentage
        percentage_below_zero = (points_below_zero / total_points) * 100

        # Print the result
        print("---------------------------------------------------------")
        print(f"Percentage of 'Phasic' values under 0: {percentage_below_zero:.2f}%")
        print("---------------------------------------------------------")



    except Exception as e:
        print(f"Error processing file ID {file_id}: {e}")
    
    print('-' * 100)


### Extracting features in a 120s window

In [ ]:
import pandas as pd
import numpy as np
import Data_Management.datamanager as DM
from EDA.edaanalysis import EdaAnalysis as EA
import os


# Define the folder path for processed EDA files
folder_path = os.path.join(os.getcwd(), "EDA_Converted")

# List of file IDs
file_ids = [
    1, 2, 3, 4, 6, 9, 10, 14, 15, 16, 20, 25, 26, 28, 29, 30, 31, 35, 36, 37, 39, 40,
    41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67,
    70, 71, 72, 73, 75, 76, 78, 81, 83, 86, 87, 88, 107, 110, 90, 93, 94, 96, 98, 99,
    101, 103, 104, 105, 106, 109, 112, 113, 114
]

sampling_rate = 1000

# Analyze each file
for ID in file_ids:
    file_name = f"ID{file_id}_processed.csv"
    file_path = os.path.join(folder_path, file_name)
    
# Create a window
window_length = 120  # sec
window_start_position = 0  # sec
window_step = 3  # sec

for ID in file_ids:
    file_name = f"ID{ID}_processed.csv"
    file_path = os.path.join(folder_path, file_name)

    print('-----------------------------------------------------------------------------------------------------------------------------------')
    print('THIS IS ID:', ID)
    data = pd.read_csv(file_path)

    # Calculate data_length for each ID
    data_length = len(data) // sampling_rate  # sec  
    print ('this is data length ----------->  ', data_length)

    data_set = pd.DataFrame()  # Result

    for window_start in range(window_start_position, data_length - window_length + 1, window_step):
        # Get data sample
        

        sample1, start_idx, stop_idx = DM.get_data_sample(data, sampling_rate, window_start, window_length)
        sample1.reset_index(drop=True, inplace=True)
        #print('this is start index -------->>>> ', start_idx)
        #print('this is stop index -------->>>> ', stop_idx)
        # List of features to be extracted
        feature_list = []  # Empty = All features
        if ID==9 or ID==86:
            features= EA.get_features(
            sample1['EDA (µS)'],
            phasic_cutoff=0.005,
            sampling_freq=sampling_rate,
            feature_list=feature_list,
            #show_plot=True,
            print_results=False
            )
            
        else :
            # Call the function
            features = EA.get_features(
            sample1['EDA (µS)'],
            phasic_cutoff=0.02,
            sampling_freq=sampling_rate,
            feature_list=feature_list,
            #show_plot=True,
            print_results=False
            )
            
        _df = pd.DataFrame([features])  # Prepare feature DataFrame to be inserted in the data_set
        data_set = pd.concat([data_set, _df], ignore_index=True)

    # Save data_set to a new CSV file for each ID
    output_file_path = f'extracted_features{ID}.csv'
    data_set.to_csv(output_file_path, index=False)

    print(f'Saved processed data set for ID {ID} to {output_file_path}')
    #print(data_set.describe())

#### SCL_Mean Normalization

In [ ]:
import pandas as pd
import os


# Define the folder path for processed EDA files
folder_path = os.path.join(os.getcwd(), "Extracted_features_120s_All_36min")

# List of file IDs
file_ids = [
    1, 2, 3, 4, 6, 10, 14, 15, 16, 20, 25, 26, 28, 29, 31, 35, 36, 37, 39, 40,
    41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67,
    70, 71, 72, 73, 75, 76, 78, 83, 86, 87, 88, 107, 110, 90, 93, 94, 96, 98, 99,
    101, 103, 104, 105, 106, 109, 112, 114
]

# Loop through each ID and read the data
for ID in file_ids:
    file_name = f"extracted_features{ID}.csv"
    file_path = os.path.join(folder_path, file_name)

    print(f'Processing file: extracted_features{ID}.csv')
    
    # Construct the filename
    df = pd.read_csv(file_path)
    # Normalize the SCL_Mean column using SCL_Min and SCL_Max
    if 'SCL_Min' in df.columns and 'SCL_Max' in df.columns and 'SCL_Mean' in df.columns:
        df['SCL_Mean'] = (df['SCL_Mean'] - df['SCL_Min']) / (df['SCL_Max'] - df['SCL_Min'])
    
    df.to_csv(f'extracted_features{ID}.csv', index=False)
    print(f'Saved file: extracted_features{ID}.csv')


## Example of Analysis

### % of HG students and LG students
Groups [0-10[ [10-16[ [16-20]

In [ ]:
# Calculate percentage of students in each grade range
bins = [0, 10, 16, 21]
labels = ["[0-10[", "[10-16[", "[16-20]"]
merged_df['Grade Group'] = pd.cut(merged_df['note_finale'], bins=bins, labels=labels, right=False)

# Compute percentage of students in each group
grade_distribution = merged_df['Grade Group'].value_counts(normalize=True) * 100

# Print the percentage of students in each grade group
print("\nPercentage of Students in Each Grade Group:")
print(grade_distribution)

# Plot the grade distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=grade_distribution.index, y=grade_distribution.values, palette='viridis')
plt.xlabel("Grade Group")
plt.ylabel("Percentage of Students")
plt.title("Percentage of Students in Each Grade Group")
plt.show()


#### Strssed Before the Exam

In [ ]:

# Compute number of stressed students in each group
stressed_students_per_group = merged_df[merged_df['stressé_maintenant_début'].isin([4,5,3])]['Grade Group'].value_counts()

# Compute percentage of stressed students in each grade group
stressed_percentage_per_group = (stressed_students_per_group / total_students_per_group) * 100

# Print the percentage of stressed students in each grade group
print("\nPercentage of Stressed Students in Each Grade Group:")
print(stressed_percentage_per_group.fillna(0))

# Merge the data into a single DataFrame for better visualization
distribution_df = pd.DataFrame({
    'Total Students': total_students_per_group,
    'Stressed Students': stressed_students_per_group,
    'Stressed Percentage': stressed_percentage_per_group
}).fillna(0)

# Plot the stressed percentage distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=distribution_df.index, y=distribution_df['Stressed Percentage'], palette='viridis')
plt.xlabel("Grade Group")
plt.ylabel("Percentage of Stressed Students")
plt.title("Percentage of Stressed Students in Each Grade Group (Before The Exam)")
plt.show()


#### Tired Brfore the Exam

In [ ]:
# Compute number of stressed students in each group
stressed_students_per_group = merged_df[merged_df['fatigué_maintenant_début'].isin([4,5,3])]['Grade Group'].value_counts()

# Compute percentage of stressed students in each grade group
stressed_percentage_per_group = (stressed_students_per_group / total_students_per_group) * 100

# Print the percentage of stressed students in each grade group
print("\nPercentage of Tired Students in Each Grade Group:")
print(stressed_percentage_per_group.fillna(0))

# Merge the data into a single DataFrame for better visualization
distribution_df = pd.DataFrame({
    'Total Students': total_students_per_group,
    'Tired Students': stressed_students_per_group,
    'Tired Percentage': stressed_percentage_per_group
}).fillna(0)

# Plot the stressed percentage distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=distribution_df.index, y=distribution_df['Tired Percentage'], palette='viridis')
plt.xlabel("Grade Group")
plt.ylabel("Percentage of Tired Students")
plt.title("Percentage of Tired Students in Each Grade Group (Before The Exam)")
plt.show()


#### Variation of EDA features (Oral VS Ecrit) for different groups

##### STRESSED/ NOT STRESSED GROUPS

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path for the merged dataset
file_path_final = os.path.join(os.getcwd(), "EDA&OtherDATA.csv")

# Load the final merged dataset
df_final = pd.read_csv(file_path_final)

# Define stress group (people who answered 3, 4, or 5 in stressé_maintenant_fin)
df_final['Stress_Group'] = df_final['stressé_maintenant_début'].apply(lambda x: 'Stressed' if x in [3, 4, 5] else 'Not Stressed')

# Select EDA features for visualization
eda_features = [col for col in df_final.columns if "ORAL" in col or "ECRIT" in col]

# Compute mean values for each stress group
mean_values = df_final.groupby('Stress_Group')[eda_features].mean()

# Plot mean EDA features for Stressed vs Not Stressed groups
plt.figure(figsize=(12, 6))
mean_values.T.plot(kind='bar', figsize=(12, 6), colormap='viridis')
plt.xlabel("EDA Features")
plt.ylabel("Mean Value")
plt.title("Mean EDA Features for Stressed vs Not Stressed Groups")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Stress Group")
plt.grid()
plt.show()


##### Tired/ NOT Tired Groups

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path for the merged dataset
file_path_final = os.path.join(os.getcwd(), "EDA&OtherDATA.csv")

# Load the final merged dataset
df_final = pd.read_csv(file_path_final)

# Define stress group (people who answered 3, 4, or 5 in stressé_maintenant_fin)
df_final['Stress_Group'] = df_final['fatigué_maintenant_début'].apply(lambda x: 'TIRED' if x in [3, 4, 5] else 'Not TIRED')

# Select EDA features for visualization
eda_features_oral = [col for col in df_final.columns if "ORAL" in col]
eda_features_ecrit = [col for col in df_final.columns if "ECRIT" in col]

# Compute mean values for each stress group
mean_values_oral = df_final.groupby('Stress_Group')[eda_features_oral].mean()
mean_values_ecrit = df_final.groupby('Stress_Group')[eda_features_ecrit].mean()

# Plot bar charts for each feature separately
for oral_feature in eda_features_oral:
    ecrit_feature = oral_feature.replace("ORAL", "ECRIT")
    if ecrit_feature in eda_features_ecrit:
        plt.figure(figsize=(8, 5))
        mean_data = pd.DataFrame({
            'Feature Type': ['ORAL', 'ECRIT'],
            'Not TIRED': [mean_values_oral.loc['Not TIRED', oral_feature], mean_values_ecrit.loc['Not TIRED', ecrit_feature]],
            'TIRED': [mean_values_oral.loc['TIRED', oral_feature], mean_values_ecrit.loc['TIRED', ecrit_feature]]
        })
        mean_data.set_index('Feature Type').plot(kind='bar', figsize=(8, 5), colormap='viridis')
        plt.xlabel("Feature Type")
        plt.ylabel("Mean Value")
        plt.title(f"Mean {oral_feature} and {ecrit_feature} for TIRED vs Not TIRED Groups")
        plt.legend(title="Stress Group")
        plt.grid()
        plt.show()


#### Question inattendue VS the rest of the exam

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define file paths
question_inattendue_path = os.path.join(os.getcwd(), "question_inattendue_means.csv")
eda_features_means_path = os.path.join(os.getcwd(), "EDA_features_means.csv")

# Load the datasets
df_question = pd.read_csv(question_inattendue_path)
df_eda = pd.read_csv(eda_features_means_path)

# Merge both datasets on ID
df_merged = pd.merge(df_question, df_eda, on="ID", suffixes=("_Question", "_EDA"))

# Select only EDA feature columns
eda_features = [col for col in df_eda.columns if col != "ID"]

# Plot bar charts for each feature
for feature in eda_features:
    question_values = df_merged[f"{feature}_Question"]
    eda_values = df_merged[f"{feature}_EDA"]
    ids = df_merged["ID"]

    # Create bar positions
    x = np.arange(len(ids))
    width = 0.35  # Width of the bars

    plt.figure(figsize=(12, 6))
    plt.bar(x - width/2, question_values, width, label="Question Inattendue", color="blue")
    plt.bar(x + width/2, eda_values, width, label="Rest of Exam", color="orange")

    # Formatting
    plt.xlabel("ID")
    plt.ylabel(f"Mean {feature}")
    plt.title(f"Comparison of {feature} in Question Inattendue vs Rest of Exam")
    plt.xticks(ticks=x, labels=ids, rotation=45, ha="right")  # Rotate ID labels for readability
    plt.legend()
    plt.grid(axis="y")

    # Show plot
    plt.show()

# Save the merged results to a CSV
output_file = os.path.join(os.getcwd(), "EDA_Feature_Comparison.csv")
df_merged.to_csv(output_file, index=False)
print(f"Comparison results saved in: {output_file}")


## Machine Learning

### Creating datasets to be used in the ML models   

In [ ]:
import os
import pandas as pd

# Folder path for ORAL and ECRIT files
input_folder = os.path.join(os.getcwd(), "Extracted_features_120s_All_36min", "after normalization_34min")

# List of file IDs to process
file_ids = [
    1, 2, 3, 4, 6, 10, 14, 15, 16, 20, 25, 26, 28, 29, 31, 35, 36, 37, 39, 40,
    41, 42, 43, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 59, 60, 62, 63, 64, 66, 67,
    70, 71, 72, 73, 75, 76, 78, 83, 86, 87, 88, 107, 110, 90, 93, 94, 96, 98, 99,
    101, 103, 104, 105, 106, 109, 112, 114
]

# Features to extract statistics for
eda_features = [
    "SCR_Peak_Count","SCR_AUC", "SCL_Var", "SCR_Peak_Mean_Amplitude", "SCL_Max", 
    "SCR_MEAN", "SCL_Mean","SCR_Peak_Sum_Risetime","tonic_var"
]

# Initialize a list to store the results
data = []

# Process each file
for file_id in file_ids:
    file_path = os.path.join(input_folder, f"extracted_features{file_id}.csv")
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print("------>>  ",file_id)
        
        # Compute mean, min, max, and std for each selected feature
        row = {"ID": file_id}
        for feature in eda_features:
            if feature in df.columns:
                row[f"{feature}_mean"] = df[feature].mean()
                row[f"{feature}_min"] = df[feature].min()
                row[f"{feature}_max"] = df[feature].max()
                row[f"{feature}_std"] = df[feature].std()
        
        data.append(row)
    else:
        print(f"Warning: File {file_path} not found.")

# Convert results to a DataFrame
ml_dataset = pd.DataFrame(data)

# Load EDA_plusotherdata.csv to retrieve note_finale
eda_data_path = os.path.join(os.getcwd(), "EDAmeans_Plus_OtherDATA.csv")
if os.path.exists(eda_data_path):
    eda_df = pd.read_csv(eda_data_path)[["ID", "note_finale", "stressé_maintenant_début", "connaissances_examen", "fatigué_maintenant_début", "Moyenne de euh par minute"]]
    
    # Merge note_finale into ML dataset based on ID
    ml_dataset = ml_dataset.merge(eda_df, on="ID", how="left")
else:
    print("Warning: EDA_plusotherdata.csv not found.")

# Save the dataset
output_file = os.path.join(os.getcwd(), "ML_dataset_5.csv")
ml_dataset.to_csv(output_file, index=False)

print(f"ML dataset saved to {output_file}")

### Data Augmentation

In [ ]:
from sdv.sequential import PARSynthesizer

def generate_synthetic_data_par(metadata, df, n_samples=100):
    model = PARSynthesizer(
        metadata=metadata,
        context_columns=['note_finale'],
        verbose=True,
        epochs=256,
        )

    model.fit(df)
    # store the model to disk in a pickle file
    model.save('models.pkl')

    synthetic_data = model.sample(n_samples)

    synthetic_data.to_csv('Synthetic_data_INITIAL.csv', index=False)


In [ ]:
from sdv.metadata import SingleTableMetadata

def generate_and_validate_metadata_par(df):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=df)
    metadata.update_column(column_name='note_finale', sdtype='numerical')
    #define the subject as identifier
    metadata.update_column(column_name='ID', sdtype='id')
    #define the entities, i.e., the columns that identify the unique time-series in the data, the entities do not have a relatisionship between them
    metadata.set_sequence_key(column_name='ID')
    #set the sequence index, i.e., the column that identifies the time in the time-series
    #metadata.set_sequence_index(column_name='t_sec')
    metadata.validate()
    return metadata

In [ ]:
from sdv.metadata import SingleTableMetadata

def generate_and_validate_metadata_par(df):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=df)
    metadata.update_column(column_name='note_finale', sdtype='numerical')
    #define the subject as identifier
    metadata.update_column(column_name='ID', sdtype='id')
    #define the entities, i.e., the columns that identify the unique time-series in the data, the entities do not have a relatisionship between them
    metadata.set_sequence_key(column_name='ID')
    #set the sequence index, i.e., the column that identifies the time in the time-series
    #metadata.set_sequence_index(column_name='t_sec')
    metadata.validate()
    return metadata

In [ ]:
#iterate over the dataframes and generate synthetic data

metadata = generate_and_validate_metadata_par(combined_df)
generate_synthetic_data_par(metadata, combined_df)

In [ ]:
metadata = generate_and_validate_metadata_par(combined_df)

synthetic_data = pd.read_csv('Synthetic_data_INITIAL.csv')
df_features = combined_df

from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_features,
    synthetic_data=synthetic_data,
    metadata=metadata
)

In [ ]:
quality_report.get_details(property_name='Column Shapes')

### REAL data + SYNTHETIC data

In [ ]:
import pandas as pd

# Read the first dataset
first_dataset_filename = 'ML_dataset_5.csv'
first_df = pd.read_csv(first_dataset_filename)

# Read the second dataset
second_dataset_filename = 'ML_SYNTHETICdataset_5.csv'
second_df = pd.read_csv(second_dataset_filename)

# Concatenate the two datasets
combined_df = pd.concat([first_df, second_df], ignore_index=True)

# Optionally, save the concatenated dataset to a new CSV file
combined_dataset_filename = 'R+Sdata_5.csv'
combined_df.to_csv(combined_dataset_filename, index=False)

print("Datasets have been concatenated and saved to", combined_dataset_filename)


combined_df.head(100)

### Prediction

Erreur Quadratique Moyenne (MSE - Mean Squared Error)
Interprétation : Moyenne des carrés des erreurs.
Avantage : Met plus de poids sur les grandes erreurs (car élève au carré).
Inconvénient : Pas directement interprétable dans l’unité de la variable cible.

Racine de l'Erreur Quadratique Moyenne (RMSE - Root Mean Squared Error)
Interprétation : Comme le MSE mais avec une unité cohérente avec la variable cible.
Avantage : Plus facile à interpréter que le MSE.


Coefficient de Détermination (R² - R-Squared)
Interprétation : Explique quelle proportion de la variance de la variable cible est expliquée par le modèle.
  proche de 1 → Bon modèle ; 
  proche de 0 → Modèle faible.


In [ ]:
#implement a series of machine learning regressors to predict the grade of a student based on EDA information
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold


dataset_filename = 'optimalR+S.csv'
df_final = pd.read_csv(dataset_filename)

#drop the student ID
df_final = df_final.drop(columns=['ID'])
df_final = df_final.drop(columns=['préparation_examen'])

#split the data into features and target
X = df_final.drop(columns=['note_finale'])
y = df_final['note_finale']
#normalize the target
y = y/y.max()

#split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### A linear regression model

In [ ]:
#implement a linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

#predict the target
y_pred_train= lin_reg.predict(X_train_scaled)
y_pred = lin_reg.predict(X_test_scaled)


print("The model performance for training set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_train, y_pred_train)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_train, y_pred_train)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_train)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)
print("\n")


print("The model performance for testing set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)


#### A random forest regressor

In [ ]:
#implement a random forest regressor
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_scaled, y_train)

#predict the target
y_pred = rf_reg.predict(X_test_scaled)
y_train_pred= rf_reg.predict(X_train_scaled)

print("The model performance for training set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_train, y_pred_train)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_train, y_pred_train)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_train)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)
print("\n")


print("The model performance for testing set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)


In [ ]:
#implement a random forest regressor with grid search

param_grid = {'n_estimators': [40, 80, 100, 140, 200, 300],
                'max_features': [None, 'sqrt', 'log2'],
                'max_depth': [15, 10, 20, 25, 30],
                'min_samples_split': [2, 5 ,6],
                'min_samples_leaf': [1, 2, 4],}
rf_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(rf_reg, param_grid, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True, n_jobs=-1, verbose=True)
grid_search.fit(X_train_scaled, y_train)

#predict the target
y_pred = grid_search.predict(X_test_scaled)

#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)

grid_search.best_params_

In [ ]:
#plot the feature importances
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.title('Feature Importances')
plt.show()

#### A gradient boosting regressor

In [ ]:
#implement a gradient boosting regressor
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(X_train_scaled, y_train)

#predict the target
y_pred = gb_reg.predict(X_test_scaled)
y_train_pred= gb_reg.predict(X_train_scaled)

print("The model performance for training set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_train, y_pred_train)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_train, y_pred_train)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_train)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)
print("\n")


print("The model performance for testing set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)

In [ ]:
#implement a gradient boosting regressor with grid search
param_grid = {'n_estimators': [200, 300, 400], 
              'max_depth': [2, 3, 4],
              'learning_rate': [0.01, 0.1, 0.5],
              'subsample': [0.8, 0.9, 1.0],
              'max_features': [0.8, 0.9, 1.0]}
gb_reg = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(gb_reg, param_grid, cv=5, scoring='neg_root_mean_squared_error', return_train_score=True)
grid_search.fit(X_train_scaled, y_train)

#predict the target
y_pred = grid_search.predict(X_test_scaled)

#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)


In [ ]:
grid_search.best_params_

In [ ]:
#plot the feature importances
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.title('Feature Importances')
plt.show()

#### A support vector regressor

In [ ]:
#implement a support vector regressor
svr = SVR()
svr.fit(X_train_scaled, y_train)

#predict the target
y_pred = svr.predict(X_test_scaled)
y_train_pred= svr.predict(X_train_scaled)

print("The model performance for training set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_train, y_pred_train)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_train, y_pred_train)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_train)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)
print("\n")


print("The model performance for testing set")
print("--------------------------------------")
#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)

 #### A multi-layer perceptron regressor

In [ ]:
#implement a multi-layer perceptron regressor
from sklearn.neural_network import MLPRegressor
mlp_reg = MLPRegressor(random_state=42, max_iter=100000, hidden_layer_sizes=(128, 128))
mlp_reg.fit(X_train_scaled, y_train)

#predict the target
y_pred = mlp_reg.predict(X_test_scaled)

#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)

In [ ]:
#implement a multi-layer perceptron regressor with grid search
param_grid = {'hidden_layer_sizes': [(64, 64), (128, 128), (256, 256)],
              'alpha': [ 0.001, 0.01, 0.1, 1, 10]}
mlp_reg = MLPRegressor(random_state=42, max_iter=100000)
grid_search = GridSearchCV(mlp_reg, param_grid, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
grid_search.fit(X_train_scaled, y_train)

#predict the target
y_pred = grid_search.predict(X_test_scaled)

#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)

grid_search.best_params_

#### A voting regressor

In [ ]:
# implement a voting regressor
from sklearn.ensemble import VotingRegressor
voting_reg = VotingRegressor(estimators=[('rf', rf_reg), ('gb', gb_reg), ('mlp', mlp_reg)])
voting_reg.fit(X_train_scaled, y_train)

#predict the target
y_pred = voting_reg.predict(X_test_scaled)

#calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

#calculate rmse score
rmse = np.sqrt(mse)
print('RMSE:', rmse)

#calculate the R^2 score
r2 = r2_score(y_test, y_pred)
print('R^2:', r2)

#compute the Akaike Information Criterion (AIC)
n = len(y_test)
k = len(X_test.columns)
aic = n * np.log(mse) + 2 * k
print('AIC:', aic)